In [5]:
import numpy as np
import pandas as pd


In [17]:
def LoadData_():
    """
    Load data set 
    
    Return:
    ------
        data: DataFrame.
        
    Note:
    ----
        last column: labels
    """
    datasets = np.array([[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3],
                  ['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L'],
                  [-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]]).T   ####  str(字符串)>float（浮点）>int（整形）
    columns = np.array(['X1','X2','Y'])

    data = pd.DataFrame(data=datasets,columns=columns)
    return data

In [22]:
data = LoadData_()

In [76]:
def calc_xianyan(data):
    labels = data.iloc[:,-1]
    all_size = labels.index.size
    
    dict_ = {}
    for i in labels:
        if i in dict_:
            dict_[i] += 1. / all_size 
        else:
            dict_[i] = 1. / all_size 
        
   
    return dict_

In [84]:
def calc_tiaojian(data,test_data):
    all_size = data.index.size
    Features = data.columns[:-1]      ##for feature in test_data:  feature=2
    
    P_I = 1
    for i in range(len(test_data)):
        sum_feature = (data[Features[i]] == test_data[i]).sum()    ###分子部分
        p_i = sum_feature + 1e-10/ all_size 
        
        P_I *= p_i
    return P_I

In [85]:
def split_data(data,test_data):
    p_y = calc_xianyan(data)
    labels = pd.unique(data.iloc[:,-1])   ###子特征拆分
    ALL_P_I = []
    for label in labels:
        p_y_xianyan = p_y[label]
        splitData = data[data.iloc[:,-1] == label]
        P_I = calc_tiaojian(splitData,test_data)       ####条件概率
        ALL_P_I.append([label,np.log(P_I * p_y_xianyan)])
       
        
    predict = sorted(ALL_P_I,key=lambda z:z[1],reverse=True)[0][0]
   
    return predict

In [86]:
test_data = ['2','S']
split_data(data,test_data)

'-1'

In [83]:
def NaiveBayes(data,test_X,gamma,is_print=False):
    """
    Build Naive Bayes
    
    Parameters:
    ----------
        data: training set.
        test_X: testing set.
        gamma: Laplace smoothing
        is_print: is print label category and pos posterior probability.
    """
    
    # 获取labels
    labels = data.iloc[:,-1]
    # 获取labels的类别个数K
    labels_ = np.array(list(set(labels)))
    len_labels_ = labels_.shape[0]
    
    # 先验概率
    pri_prob= np.zeros((len_labels_))
    
    
    # 预测值保留数组
    m,n = test_X.shape
    cache_predict = np.zeros((m,len_labels_))
    
    # 计算先验概率
    for i in range(len_labels_):
        P_y = (labels[labels == labels_[i]].size + gamma) / (labels.size + len_labels_ * gamma)
        pri_prob[i] = P_y
    
    # 计算条件概率
    for i in range(m):
        for j in range(len_labels_):
            Conditional_Prob = 1
            for k in range(n):
        
                data_label = data[labels==labels_[j]] # 该标签下的所有数据
                future_k = data_label.iloc[:,k]  # test_X 的第k个特征列
                molecule = data_label[future_k == test_X[i,k]].shape[0] + gamma # 分子部分
                Sj = len(set(future_k))
                denominator = data_label.shape[0] + (Sj * gamma) # 分母部分
                
                Conditional_Prob *= molecule /denominator # 计算条件概率
                
            Pos_proba = pri_prob[j] * Conditional_Prob # 计算后验概率
            
            cache_predict[i,j] += Pos_proba
        
    # if ture,print labels_ and predict probability array.
    if is_print:
        print(labels_)
        print(cache_predict)
        
    best_predict_index = np.argmax(cache_predict,axis=1)
    return labels_[best_predict_index]

In [67]:
test_x = np.array([['2','S']])
predic_label = NaiveBayes(data=data,test_X=test_x,gamma=1,is_print=True)
print('Predict label is: ',predic_label)

['1' '-1']
[[0.03267974 0.06100218]]
Predict label is:  ['-1']
